In [1]:
import os
import librosa, librosa.display
import matplotlib.pyplot as plt
import math
import numpy as np
import IPython.display as ipd
import matplotlib.pyplot as plt
import librosa.display
import tensorflow as tf
from tensorflow import keras
from keras import layers
from keras import models
from keras.utils import to_categorical
import pandas as pd

2023-03-02 13:37:47.205999: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-02 13:37:47.575798: I tensorflow/core/util/port.cc:104] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-03-02 13:37:47.649179: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-02 13:37:47.649193: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore 

In [2]:
direc = os.getcwd()

In [3]:
def dictionarize_data (directory): #"D:\TIMIT\TRAIN" "No linux é /media/roberto/UFD/TIMIT/TRAIN"
    data = {"signal_path" : [],
            "phn_path" : [],
            "wrd_path" : [],
            "txt_path" : []}
    for root, direc, file in os.walk(directory):
            for i in file:            
                ext = i.split('.')
                file_path = os.path.join(root, ext[0])
                wav = "WAV"
                if ext[1] == wav:
                    data["signal_path"].append(file_path + ".WAV")
                    data["phn_path"].append(file_path + ".PHN")
                    data["wrd_path"].append(file_path + ".WRD")
                    data["txt_path"].append(file_path + ".TXT")
    return data   

In [4]:
def num_labels(lim_vec,label_vec,beg,end): #resolve o problema de intervalos inter-rotulos
    i = 0
    while i < len(lim_vec) and not(lim_vec[i][0] <= beg <= lim_vec[i][1]):
        i+=1
    j = 0
    
    while j < len(lim_vec) - 1 and not(lim_vec[j][0] <= end <= lim_vec[j][1]):
        j+=1
    return i,j

In [5]:
def save_melspectrogram(janelas_vec, sample_rate, nome, path_janela): #"/home/roberto/iniciacao_cientifica/JANELAS"
    n_fft = 2048
    janela = 512

    #calcula a short term fourier transform:
    stft = librosa.stft(janelas_vec, n_fft = n_fft, hop_length = janela)

    #calcula o valor absoluto da stft (spectogram):
    spectrogram = np.abs(stft)

    #calcula o melspectrogram:
    melspectrogram = librosa.feature.melspectrogram(y = janelas_vec, S = spectrogram ,sr = sample_rate, hop_length = janela, fmax = sample_rate/2)
    file_name = str(nome) 
    os.chdir(path_janela)
    diretorio = os.getcwd()
    np.save(file_name,np.array(melspectrogram))
    path = os.path.join(diretorio, file_name)
    return path

In [6]:
def save_labels(labels_vec, path): #"/home/roberto/iniciacao_cientifica/ROTULOS"
    os.chdir(path) 
    np.save("rotulos.npy",labels_vec)


In [7]:
def divide_signal(data,jan_path,label_path):   
    #divide o sinal em intervalos de 100 ms (com passo de ms) da um rotulo -- caso o valor não exista, preencher com 0 até que de 100 ms
    #prestar atenção à valores de intervalo que ficam entre dois (ou mais) rotulos
    #copiar duas vezes e dar os dois (ou mais) rotulos
    #fazer a transformada de fourier e tmb melspectrograma nessa função mesmo!!
    janelas_vec = []
    labels_vec = []
    cont = 0
    window_data = {"window_path": [],
                   "label":[]}
    result = "resultado"
    
    for values in data["phn_path"]:
        limites_vec = []
        fonemas_vec = []
        #carrega o arquivo .WAV do correspondente ao arquivo .PHN
        path = values.split(".")
        #print(path[0] + ".WAV")
        raw_signal, sample_rate = librosa.load(path[0] + ".WAV", sr = 22050) #sr a definir

        #permite a leitura do arquivo phn
        read = open(values, "r")

        #guarda em um vetor todas as linhas do arquivo .PHN
        lines = read.readlines()

        #define o valor limite que possui rótulos
        lin = lines[-1].strip()
        modules = lin.split(" ")
        limite_total = int(modules[1]) # após esse valor não existe mais rótulos

        #copia o vetor do sinal extraido do .WAV do arquivo até o limite total
        signal = raw_signal[:limite_total]

        #adiciona todos os fonemas(rótulos) e seus intervalos em uma lista
        for line in lines: 
            li = line.strip()
            data_phonema = li.split(" ")
            lim_chao = int(data_phonema[0]) #limite por baixo
            lim_teto = int(data_phonema[1]) #limite por cima
            fonema = data_phonema[2]
            limites = (lim_chao,lim_teto)
            limites_vec.append(limites)
            fonemas_vec.append(fonema)
        cont_2 = 0
        #separa o vetor em vetores menores (janelas copias)
        for i in range(0, limite_total, int(sample_rate / 20)): 
            end = i + int(sample_rate/10)
            
            if end < limite_total:
                janela = signal[i : end]
            else:
                janela = signal[i : limite_total-1]
                
            while len(janela) < (2**math.ceil(math.log(sample_rate/10, 2))): #se o tamanho do vetor for menor que a proxima potencia acima dele, adiciona 0 
                janela = np.append(janela, 0)
                
            inicio, final = num_labels(limites_vec, labels_vec, i,end) #registra a primeira e a ultima ocorrencia da janela
            
            vers = 0
            for j in range(inicio, final + 1):
                jan = np.array(janela)
                wind_path = save_melspectrogram(jan, sample_rate, (result +"_s"+str(cont) + "_j"+str(cont_2)+"_v"+str(vers)),jan_path)
                os.chdir(direc)
                window_data["window_path"].append(wind_path)
                window_data["label"].append(fonemas_vec[j])
                vers +=1
            cont_2 += 1
        cont += 1
    save_labels(window_data["label"],label_path)
    os.chdir(direc)        
    return window_data

In [8]:
def load_from_path(path_janela, path_rotulo): #carrega os paths e rotulos das janelas já pré-processadas
    janela_vec = []
    for root, direc, file in os.walk(path_janela):
        for i in file:
            janela_vec.append(i)
    rotulos_vec = np.load(path_rotulo)
    return janela_vec, rotulos_vec

In [9]:
def contains(vec, label):
    for v in vec:
        if v == label:
            return True
    return False

In [10]:
def lista_labels(window_dict):
    lista_labels = []
    for i in window_dict["label"]:
        if contains(lista_labels,i) == False:
            lista_labels.append(i)
        else:
            continue
    return lista_labels


In [11]:
def classifica_label(label):
    if label in ['ae', 'ao', 'aa', 'ah']:
        label = 'a'
    elif label in ['eh', 'ey', 'iy', 'er']:
        label = 'e'
    elif label in ['ow', 'oy', 'aw', 'uw','axr']:
        label = 'o'
    elif label in ['ih', 'ay','ix']:
        label = 'i'
    elif label in ['uh', 'ux','uw']:
        label = 'u'
    else:
        label = 'consonant'
    return label


In [12]:
def classifica_lista(lista):
    resultado = []
    for i in lista:
        label = classifica_label(i)
        resultado.append(label)
    return resultado

In [13]:
train_window, train_label = load_from_path("/home/roberto/iniciacao_cientifica/JANELAS","/home/roberto/iniciacao_cientifica/ROTULOS/rotulos.npy")
window_dict= {"window_path": [],
                   "label":[]}
window_dict["window_path"] = train_window[:]
window_dict["label"] = train_label[:]
train_classified_labels = classifica_lista(window_dict["label"])
window_dict["label"] = train_classified_labels

In [19]:
dictionary = exclui_consoantes(window_dict)
window_dict = dictionary


In [31]:
test_window, test_label = load_from_path("/home/roberto/iniciacao_cientifica/JANELAS_TESTE","/home/roberto/iniciacao_cientifica/ROTULOS_TESTE/rotulos.npy")
window_dict_test = {"window_path": [],
                   "label":[]}
window_dict_test["window_path"] = test_window[:]
window_dict_test["label"] = test_label[:]
test_classified_labels = classifica_lista(window_dict_test["label"])
window_dict_test["label"] = test_classified_labels

In [32]:
dict_test = exclui_consoantes(window_dict_test)
window_dict_test = dict_test


In [15]:
def count_labels(labels):
    vec = []
    contador = 0
    for i in labels:
        if not contains(vec,i):
            vec.append(i)
            contador += 1
    print(vec)
    return contador

In [16]:
def normalize_pixels(data):
    #pixels ficam entre 0 e 1
    data = data.astype('float32')/255.0
    return data

In [17]:
def categorize_labels(label):
    label = pd.get_dummies(label)
    return label    

In [18]:
def exclui_consoantes(window_dict):
    window_data = {"window_path": [],
                   "label":[]}
    contador = 0
    for i in window_dict["label"]:
        if i != "consonant":
            window_data["window_path"].append(window_dict["window_path"][contador])
            window_data["label"].append(window_dict["label"][contador])
        contador += 1
    return window_data
        

In [20]:
def load_data(window_data, path):
    t_data = np.array(window_data["window_path"])
    train_data = []
    for t in t_data:
        train_data.append(np.load(path+"/"+t))
    train_data = np.array(train_data)
    train_label = window_data["label"]
    return train_data, train_label

In [21]:
def create_network():
    network = models.Sequential()
    # network layers:
    network.add(layers.Conv1D(32, 3, activation='relu', padding='same', kernel_initializer='he_uniform', input_shape=(128, 9)))
    network.add(layers.Conv1D(32, 3, activation='relu', padding='same', kernel_initializer='he_uniform'))
    network.add(layers.MaxPooling1D(2))
    network.add(layers.Conv1D(64, 3, activation='relu', padding='same', kernel_initializer='he_uniform'))
    network.add(layers.Conv1D(64, 3, activation='relu', padding='same', kernel_initializer='he_uniform'))
    network.add(layers.MaxPooling1D(2))
    network.add(layers.Conv1D(128, 3, activation='relu', padding='same', kernel_initializer='he_uniform'))
    network.add(layers.Conv1D(128, 3, activation='relu', padding='same', kernel_initializer='he_uniform'))
    network.add(layers.MaxPooling1D(2))
    network.add(layers.Flatten())
    network.add(layers.Dense(128, activation='relu'))
    network.add(layers.Dense(5, activation='softmax'))
    # network configurations (optimizer, loss and metrics functions):
    network.compile(optimizer='RMSprop',
                    loss='categorical_crossentropy',
                    metrics=['accuracy'])
    return network



In [22]:
train_data, train_label = load_data(window_dict, "/home/roberto/iniciacao_cientifica/JANELAS")
train_data = normalize_pixels(train_data)
train_label = categorize_labels(train_label)

network = create_network()
network.fit(train_data, train_label, epochs = 5, batch_size = 128)



2023-03-02 13:38:37.099338: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2023-03-02 13:38:37.099736: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2023-03-02 13:38:37.099749: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (roberto-Inspiron-15-3511): /proc/driver/nvidia/version does not exist
2023-03-02 13:38:37.101777: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/5
1437/1437 [==============================] - 30s 20ms/step - loss: 1.4847 - accuracy: 0.2775
Epoch 2/5
1437/1437 [==============================] - 31s 21ms/step - loss: 1.4830 - accuracy: 0.2806
Epoch 3/5
1437/1437 [==============================] - 42s 29ms/step - loss: 1.4827 - accuracy: 0.2816
Epoch 4/5
1437/1437 [==============================] - 59s 41ms/step - loss: 1.4826 - accuracy: 0.2817
Epoch 5/5
1437/1437 [==============================] - 61s 42ms/step - loss: 1.4826 - accuracy: 0.2824


In [33]:
test_data, test_label = load_data(window_dict_test, "/home/roberto/iniciacao_cientifica/JANELAS_TESTE")
test_data = normalize_pixels(test_data)
test_label = categorize_labels(test_label)

loss, accuracy = network.evaluate(test_data, test_label)

print('Test loss:', loss)
print('Test accuracy:', accuracy)

2091/2091 [==============================] - 16s 7ms/step - loss: 1.4760 - accuracy: 0.2949
Test loss: 1.4760438203811646
Test accuracy: 0.29488980770111084
